# MoSDeF Overview

To begin, we need to import the `mbuild` package, here using the alias `mb`. This will give us access to all of the data structures and functions within mBuild.

In [ ]:
import mbuild as mb

Now we'll load a CH2 moiety into an mBuild Compound by reading from a PDB structure file (created using [Avogadro](https://avogadro.cc/)). This will create an mBuild `Compound` containing three atoms (C, H, H), as well as two C-H bonds.

In [ ]:
ch2 = mb.load('ch2.pdb')

mBuild has a built-in `Compound.visualize` method for use in Jupyter notebooks.

In [ ]:
ch2.visualize()

mBuild Compound's have a variety of attributes and methods (http://mosdef-hub.github.io/mbuild/data_structures.html). These can be queried using `dir(ch2)`.

In [ ]:
dir(ch2)

One of the methods of mBuild `Compounds` is `particles()`, which is a generator that returns all of the `Particles` within a `Compound`.

In [ ]:
list(ch2.particles())

Another method, `particles_by_name` returns only `Particles` with a specified name. Here we will use the `particles_by_name` method to grab the carbon atom in our `Compound`.

In [ ]:
carbon = list(ch2.particles_by_name('C'))[0]
carbon

The `Port` class (http://mosdef-hub.github.io/mbuild/data_structures.html#mbuild.port.Port) is used to create ports, essentially dangling bonds, that can be used to connect `Compound`'s together and can accept several helpful arguments upon instantiation to orient the port in the desired direction.

Here we will create two ports to add to our CH2 moiety to represent the two dangling bonds. We will orient these in opposite directions (+*y* and -*y*) and shift these from the carbon position by half of a C-C bond length
- **Note**: mBuild uses _nanometers_ as the distance unit.

In [ ]:
up_port = mb.Port(anchor=carbon, orientation=[0, 1, 0], separation=0.075)
down_port = mb.Port(anchor=carbon, orientation=[0, -1, 0], separation=0.075)
up_port

Now we can use the `add` function to add the ports we've just created to our `Compound`. `add` can be used to add `Compound`'s to other `Compound`'s.

In [ ]:
ch2.add(up_port, label='up')
ch2.add(down_port, label='down')
ch2.available_ports()

Any part added to a `Compound` can be given an optional, descriptive string `label` for easy referencing. In the above code block, we added the labels `up` and `down` to the two `Port`'s that we added to our CH2 moiety.

In [ ]:
ch2['up']

Let's visualize the ch2 `Compound` again, this time showing the two `Ports` we've added.

In [ ]:
ch2.visualize(show_ports=True)

The mBuild `clone` function is an alternative to `deepcopy` and allows us to create copies of `Compound`'s. (If you are unfamiliar with the concept of a deep copy, here is a description copied from the Python manual: *A deep copy constructs a new compound object and then, recursively, inserts copies into it of the objects found in the original.*)

In [ ]:
ch2_copy = mb.clone(ch2)

The `force_overlap` function takes a `Compound` and then rotates and translates it such that two other `Compounds` overlap. Typically, as in the following case, those two other `Compounds` are `Ports` - in our case, `ch2_copy['up']` and `ch2['down']`.

In [ ]:
mb.force_overlap(move_this=ch2_copy,
                 from_positions=ch2_copy['up'],
                 to_positions=ch2['down'])

We now have two CH2 moieties bonded together. Next, we'll add these two `Compounds` to a parent `Compound` that we will call `alkane`, which we can visualize or otherwise manipulate.

In [ ]:
alkane = mb.Compound()
alkane.add(ch2, label='ch2[$]')
alkane.add(ch2_copy, label='ch2[$]')

In [ ]:
alkane.visualize(show_ports=True)

In [ ]:
alkane

It would be cumbersome to have to create `Compounds` by hand each time we wanted to use them. Instead, mBuild is intended to be used in such a way that reusable classes are defined for various `Compounds`. Here, we'll define a class for our CH2 moiety.

In [ ]:
class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        mb.load('ch2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        self.add(mb.Port(anchor=carbon, orientation=[0, 1, 0], separation=0.075), 'up')
        self.add(mb.Port(anchor=carbon, orientation=[0, -1, 0], separation=0.075), 'down')

If we instantiate this class and visualize we should see the same result we obtained earlier.

In [ ]:
ch2 = CH2()
ch2.visualize(show_ports=True)

Now that we have a class defined for our CH2 moiety, we can utilize this to create an alkane chain. Here we'll define a class for an alkane chain that can take `chain_length` as an argument. We will achieve this by successively adding CH2 moieties, and capping the first and last moieties with hydrogen atoms.

In [ ]:
from mbuild.lib.atoms import H

class Alkane(mb.Compound):
    def __init__(self, chain_length=1):
        super(Alkane, self).__init__()
        last_monomer = CH2()
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['up'])
        self.add(last_monomer)
        self.add(hydrogen)
        for _ in range(chain_length-1):
            current_monomer = CH2()
            mb.force_overlap(move_this=current_monomer,
                             from_positions=current_monomer['up'],
                             to_positions=last_monomer['down'])
            self.add(current_monomer)
            last_monomer=current_monomer
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['down'])
        self.add(hydrogen)

Now that we've created an `Alkane` class, we can instantiate this and provide any chain length (>0) that we want.

In [ ]:
alkane = Alkane(chain_length=10)
alkane.visualize(show_ports=True)

One downside to creating compounds in this manner is that you often end up with non-realistic configurations (such as the alkane above with all angles at 180 degrees). This is typically resolved through an energy minimization of the system you create in whichever simulation package you are using. However, mBuild also offers a built-in energy minimization routine that will attempt to optimize a `Compound`'s geometry.

In [ ]:
alkane.energy_minimization(steps=5000)

In [ ]:
alkane.visualize()

mBuild also provides a `Polymer` recipe that essentially performs the same operations as the `Alkane` class we've defined above, but provides additional flexibility to support arbitrary monomer units as well as copolymers. Here we'll modify our `Alkane` class to use the `Polymer` recipe.

In [ ]:
class Alkane(mb.Compound):
    def __init__(self, chain_length=1):
        super(Alkane, self).__init__()
        chain = mb.Polymer(CH2(), chain_length)
        self.add(chain, label='chain')
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=chain['up'])
        self.add(hydrogen, label='up-cap')
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=chain['down'])
        self.add(hydrogen, label='down-cap')

In [ ]:
alkane = Alkane(6)
alkane.visualize()

Particles can be easily added and removed in mBuild. Here, we'll explore this functionality by changing our hexane molecule into _hexanol_.

First, we'll create a class for a hydroxyl group.

In [ ]:
class OH(mb.Compound):
    def __init__(self):
        super(OH, self).__init__()
        self.add(mb.Particle(name='O', pos=[0.0, 0.0, 0.0]), label='O')
        self.add(mb.Particle(name='H', pos=[0.0, 0.1, 0.0]), label='H')
        self.add_bond((self['O'], self['H']))
        self.add(mb.Port(anchor=self['O'], orientation=[0, -1, 0], separation=0.075), label='down')

Let's visualize to see if everything looks good.

In [ ]:
hydroxyl = OH()
hydroxyl.visualize(show_ports=True)

Now we'll remove a hydrogen from one end of our hexane and add a hydroxyl.

Below, we remove the hydrogen compound that has the label `up-cap`.
We see that there is a single port available.

In [ ]:
alkane.remove(alkane['up-cap'])
ports = alkane.all_ports()
ports

Next, we force the single available port to overlap with the `down` port of the hydroxyl compound.

In [ ]:
mb.force_overlap(move_this=hydroxyl,
                 from_positions=hydroxyl['down'],
                 to_positions=ports[0])
hexanol = mb.Compound()
hexanol.add(alkane, label='alkane')
hexanol.add(hydroxyl, label='hydroxyl')

We can now visualize to make sure our hexanol was successfully created.

In [ ]:
hexanol.visualize()

Running simulations of a single small molecule would likely not be very interesting. mBuild offers several routines to help create more complex systems. One of these routines is the `fill_box` function, which can be used to fill a box of a user-defined size with a specified number of molecules. We'll check this out now by placing 10 hexanols in a 3nm x 3nm x 3nm box.

In [ ]:
box = mb.fill_box(hexanol, n_compounds=10, box=[3, 3, 3], seed=2)
box.visualize()

Now if we wanted to actually run a simulation of this system we would need to apply a force field and write the necessary data files. mBuild handles all of this through a single `save` command, where we can pass as arguments the name of the force field to apply (which uses Foyer under the hood) and the name of the file to create, which will be formatted based on the extension. Here we will save in Gromacs `TOP` and `GRO` formats.

In [ ]:
box.save('system.top', forcefield_name='oplsaa', overwrite=True)
box.save('system.gro', overwrite=True)

The last bit of functionality we will observe is attaching molecules to a surface. In our case here, we first need to remove the hydrogen atom opposite of the hydroxyl so that we have an available port on our hexanol to attach to a surface.

In [ ]:
hexanol.remove(hexanol['alkane']['down-cap'])
hexanol.add(hexanol.all_ports()[0], 'down', containment=False)
hexanol.visualize(show_ports=True)

The surface we will attach our hexanes to is crystalline silica, which we will import from mBuild's compound library. In many cases it is desirable to expand `Compounds` with periodicity, like surfaces, in one or more dimensions. mBuild features a built-in routine, `TiledCompound` to handle this functionality. Here we will use `TiledCompound` to expand our surface in the *x* dimension.

In [ ]:
from mbuild.lib.surfaces import Betacristobalite
surface = Betacristobalite()
tiled_surface = mb.TiledCompound(surface, n_tiles=(2, 1, 1))

mBuild contains a `Pattern` class which can be used to create patterns to place molecules at desired locations in space. This is useful for many cases, including defining the arrangement of chains on a surface. Here, we will create a `Random2DPattern` of 10 points in *xy* space.

The `Pattern.apply_to_compound` method can be used to attach `Compounds` to another `Compound` (e.g. a surface) by finding the vacant ports in the `host` `Compounds` closest to those defined by the pattern. Additionally, a `backfill` can be defined that will fill any leftover ports. Here, we will define a hydrogen atom as our backfill.

In [ ]:
pattern = mb.Random2DPattern(10)
hydrogen = H()
chains, backfills = pattern.apply_to_compound(guest=hexanol, host=tiled_surface, backfill=hydrogen)

Finally, we'll add our surface, chains, and backfills to a parent `Compound` and visualize.

In [ ]:
functionalized_surface = mb.Compound()
for part in [tiled_surface, chains, backfills]:
    functionalized_surface.add(part)
functionalized_surface.visualize()